### C

In [ ]:
n = int(input())
li = [[] for _ in range(n + 1)]

for _ in range(n - 1):
    s, e = map(int, input().split())
    li[s].append(e)
    li[e].append(s)
    


### I

In [ ]:
n, m = map(int, input().split())
xs = list(map(int, input().split()))
mails = [list(map(int, input().split())) for _ in range(m)]



In [ ]:
돌아가야 되는 사람은 자유 아닌가?`

### J

In [10]:
n = int(input())

li = list(map(int, input().split()))
sli = sorted(li)

ip = True
for i in range(n):
    temp = li[i] * sli[i]
    if temp ** 0.5 != int(round(temp ** 0.5, 5)):
        ip = False
        break
        
if ip:
    print('YES')
else:
    print('NO')

2
2 1
NO


F

In [ ]:
끊임없이 돌면서 가능한거 있으면 가져감

하나 가져가려면 최대 100이 필요함


In [20]:
from collections import defaultdict as dd
import sys
input = lambda: sys.stdin.readline().rstrip()

n, m, s_len = map(int, input().split())
li = [input() for _ in range(n)]

d = dd(list)
for i in range(n):
    for j in range(m):
        d[li[i][j]].append([i, j])
    
s = input()

r = []
temp = []
si = 0
rein = 0
sx, sy = 0, 0
x, y = 0, 0
while True:
    to_find = s[si]
    if not d[to_find]:
        nx, ny = n - 1, m - 1
        x, y = sx, sy

        while x > nx:
            x -= 1
            r.append('U')
        while x < nx:
            x += 1
            r.append('D')
        while y > ny:
            y -= 1
            r.append('L')
        while y < ny:
            y += 1
            r.append('R')
        break
        
    nx, ny = d[to_find].pop()
    
    while x > nx:
        x -= 1
        temp.append('U')
    while x < nx:
        x += 1
        temp.append('D')
    while y > ny:
        y -= 1
        temp.append('L')
    while y < ny:
        y += 1
        temp.append('R')
    temp.append('P')
    
    si += 1
    if si == s_len:
        rein += 1
        r.extend(temp)
        temp = []
        sx, sy = x, y
        si = 0
    
print(rein, len(r))
print(''.join(r))

3 3 4
icp
cuc
pci
ucpc
1 12
DRPDPLPURRPD


G

In [ ]:
1 ~ 2 * n까지 block이든 chain이든 있어야함

같은 로봇의 block이 두 번 연속 있는 경우 그 사이에 다른 로봇의
체인 연산이 무조건 있어야함

각 로봇은 n개씩의 블록을 가지고 있음

다른 로봇의 체인 연산 부분에 가장 작은 셋에 현재 넣을 수 있는 가장 작은 수를 찾아서 넣음?

In [25]:
# https://github.com/tatyam-prime/SortedSet/blob/main/SortedMultiset.py
import math
from bisect import bisect_left, bisect_right, insort
from typing import Generic, Iterable, Iterator, TypeVar, Union, List
T = TypeVar('T')

class SortedMultiset(Generic[T]):
    BUCKET_RATIO = 50
    REBUILD_RATIO = 170

    def _build(self, a=None) -> None:
        "Evenly divide `a` into buckets."
        if a is None: a = list(self)
        size = self.size = len(a)
        bucket_size = int(math.ceil(math.sqrt(size / self.BUCKET_RATIO)))
        self.a = [a[size * i // bucket_size : size * (i + 1) // bucket_size] for i in range(bucket_size)]
    
    def __init__(self, a: Iterable[T] = []) -> None:
        "Make a new SortedMultiset from iterable. / O(N) if sorted / O(N log N)"
        a = list(a)
        if not all(a[i] <= a[i + 1] for i in range(len(a) - 1)):
            a = sorted(a)
        self._build(a)

    def __iter__(self) -> Iterator[T]:
        for i in self.a:
            for j in i: yield j

    def __reversed__(self) -> Iterator[T]:
        for i in reversed(self.a):
            for j in reversed(i): yield j
    
    def __len__(self) -> int:
        return self.size
    
    def __repr__(self) -> str:
        return "SortedMultiset" + str(self.a)
    
    def __str__(self) -> str:
        s = str(list(self))
        return "{" + s[1 : len(s) - 1] + "}"

    def _find_bucket(self, x: T) -> List[T]:
        "Find the bucket which should contain x. self must not be empty."
        for a in self.a:
            if x <= a[-1]: return a
        return a

    def __contains__(self, x: T) -> bool:
        if self.size == 0: return False
        a = self._find_bucket(x)
        i = bisect_left(a, x)
        return i != len(a) and a[i] == x

    def count(self, x: T) -> int:
        "Count the number of x."
        return self.index_right(x) - self.index(x)

    def add(self, x: T) -> None:
        "Add an element. / O(√N)"
        if self.size == 0:
            self.a = [[x]]
            self.size = 1
            return
        a = self._find_bucket(x)
        insort(a, x)
        self.size += 1
        if len(a) > len(self.a) * self.REBUILD_RATIO:
            self._build()

    def discard(self, x: T) -> bool:
        "Remove an element and return True if removed. / O(√N)"
        if self.size == 0: return False
        a = self._find_bucket(x)
        i = bisect_left(a, x)
        if i == len(a) or a[i] != x: return False
        a.pop(i)
        self.size -= 1
        if len(a) == 0: self._build()
        return True

    def lt(self, x: T) -> Union[T, None]:
        "Find the largest element < x, or None if it doesn't exist."
        for a in reversed(self.a):
            if a[0] < x:
                return a[bisect_left(a, x) - 1]

    def le(self, x: T) -> Union[T, None]:
        "Find the largest element <= x, or None if it doesn't exist."
        for a in reversed(self.a):
            if a[0] <= x:
                return a[bisect_right(a, x) - 1]

    def gt(self, x: T) -> Union[T, None]:
        "Find the smallest element > x, or None if it doesn't exist."
        for a in self.a:
            if a[-1] > x:
                return a[bisect_right(a, x)]

    def ge(self, x: T) -> Union[T, None]:
        "Find the smallest element >= x, or None if it doesn't exist."
        for a in self.a:
            if a[-1] >= x:
                return a[bisect_left(a, x)]
    
    def __getitem__(self, x: int) -> T:
        "Return the x-th element, or IndexError if it doesn't exist."
        if x < 0: x += self.size
        if x < 0: raise IndexError
        for a in self.a:
            if x < len(a): return a[x]
            x -= len(a)
        raise IndexError

    def index(self, x: T) -> int:
        "Count the number of elements < x."
        ans = 0
        for a in self.a:
            if a[-1] >= x:
                return ans + bisect_left(a, x)
            ans += len(a)
        return ans

    def index_right(self, x: T) -> int:
        "Count the number of elements <= x."
        ans = 0
        for a in self.a:
            if a[-1] > x:
                return ans + bisect_right(a, x)
            ans += len(a)
        return ans

n, m = map(int, input().split())

not_vt = SortedMultiset()
for i in range(1, n * 2 + 1):
    not_vt.add(i)
    
stacks = SortedMultiset()

li = []
fv = 1
for _ in range(m):
    tp, _, b_num = input().split()
    if tp == 'A':
        r_num = 0
    else:
        r_num = 1
    
    b_num = int(b_num)
    not_vt.discard(b_num)
    if r_num == fv:
        li.append([r_num ^ 1, '?'])
        li.append([r_num, b_num])
    else:
        li.append([r_num, b_num])
        fv = r_num

for i in range(len(li)):
    
    if li[i][1] != '?':
        stacks.add(li[i][1])
    else:
        stacks[0] 위에 올릴 수 있는 가장 작은게 뭘까?

sdf


ValueError: invalid literal for int() with base 10: 'sdf'

In [ ]:
from bisect import bisect_left

li = [2, 4, 6]
bisect_left(li, )

In [ ]:
A8 A1 B3 B6 A4
A8 B? A1 B3 A? B6 A4
 
A8 B9 A1 B3 A    

8, 1, 3, 6, 4



In [ ]:
2, 5, 7, 9, 10, 11, 12